In [2]:
# [데이터 분할을 위한 폴더 생성]

import os
import shutil

original_dataset_dir = '/home/iai/Desktop/Jeewon/Study/PyTorch/PyTorchProject/data/plant'
classes_list = os.listdir(original_dataset_dir)

base_dir = '/home/iai/Desktop/Jeewon/Study/PyTorch/PyTorchProject/data/splitted'
os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# train,validation,test 폴더 하위에 각각 클래스 목록 폴더 생성
for clss in classes_list:
    os.mkdir(os.path.join(train_dir,clss))
    os.mkdir(os.path.join(validation_dir,clss))
    os.mkdir(os.path.join(test_dir,clss))

In [3]:
# [데이터 분할과 클래스별 데이터 수 확인]

import math

for clss in classes_list:
    path = os.path.join(original_dataset_dir,clss)
    fnames = os.listdir(path)
    
    train_size = math.floor(len(fnames)*0.6)
    validation_size = math.floor(len(fnames)*0.2)
    test_size = math.floor(len(fnames)*0.2)
    
    train_fnames = fnames[:train_size]
    print('Train size(',clss,'): ', len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, clss), fname)
        shutil.copyfile(src,dst)
        
    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print('Validation size(',clss,'): ', len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir,clss),fname)
        shutil.copyfile(src,dst)
        
    test_fnames = fnames[(train_size+validation_size):(validation_size+train_size+test_size)]
    print('Test size(',clss,'): ', len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, clss),fname)
        shutil.copyfile(src,dst)

Train size( Grape___healthy ):  253
Validation size( Grape___healthy ):  84
Test size( Grape___healthy ):  84
Train size( Apple___healthy ):  987
Validation size( Apple___healthy ):  329
Test size( Apple___healthy ):  329
Train size( Apple___Black_rot ):  372
Validation size( Apple___Black_rot ):  124
Test size( Apple___Black_rot ):  124
Train size( Tomato___Late_blight ):  1145
Validation size( Tomato___Late_blight ):  381
Test size( Tomato___Late_blight ):  381
Train size( Cherry___healthy ):  512
Validation size( Cherry___healthy ):  170
Test size( Cherry___healthy ):  170
Train size( Tomato___Septoria_leaf_spot ):  1062
Validation size( Tomato___Septoria_leaf_spot ):  354
Test size( Tomato___Septoria_leaf_spot ):  354
Train size( Corn___Northern_Leaf_Blight ):  591
Validation size( Corn___Northern_Leaf_Blight ):  197
Test size( Corn___Northern_Leaf_Blight ):  197
Train size( Potato___healthy ):  91
Validation size( Potato___healthy ):  30
Test size( Potato___healthy ):  30
Train si

In [8]:
# [베이스라인 모델 학습을 위한 준비]

import torch

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

BATCH_SIZE = 256
EPOCH = 30

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

transform_base = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()])

train_dataset = ImageFolder(root='/home/iai/Desktop/Jeewon/Study/PyTorch/PyTorchProject/data/splitted/train', transform = transform_base)
val_dataset = ImageFolder(root='/home/iai/Desktop/Jeewon/Study/PyTorch/PyTorchProject/data/splitted/val', transform = transform_base)


# Dataloader는 이미지 데이터를 미니 배치 단위로 분리하는 역할
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 4)

In [9]:
# [베이스라인 모델 설계]

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3,32,3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32,64,3,padding=1)
        self.conv3 = nn.Conv2d(64,64,3,padding=1)
        
        self.fc1 = nn.Linear(4096,512)
        self.fc2 = nn.Linear(512,33)
        
    def forward(self,x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training = self.training)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training = self.training)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training = self.training)
        
        x = x.view(-1, 4096)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

model_base = Net().to(DEVICE)
optimizer = optim.Adam(model_base.parameters(), lr = 0.001)

In [10]:
# [모델 학습을 위한 함수]

def train(model, train_laoder, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader): # train_loader에는 (data, target) 형태가 미니 배치 단위로 묶여 있음
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target) # 분류 문제에 적합한 cross entropy loss
        loss.backward() # back propagation을 통해 계산한 Gradient 값을 각 Parameter에 할당
        optimizer.step() # 각 Parameter에 할당된 Gradient 값을 이용해 모델의 Parameter 업데이트

In [11]:
# [모델 평가를 위한 함수]

def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad(): # torch.no_grad() : Parameter 업데이트 중단
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction = 'sum').item()
            
            pred = output.max(1, keepdim=True)[1] # Test 데이터가 33개의 클래스에 속할 각각의 확률값이 output으로 출력. 가장 높은 가진 인덱스를 예측값으로 저장
            correct += pred.eq(target.view_as(pred)).sum().item() # pred와 target.view_as(pred)가 일치하면 1, 일치하지 않으면 0 반환
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [13]:
# [모델 학습 실행]

import time
import copy

def train_baseline(model, train_loader, val_loader, optimizer, num_epochs = 30):
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict()) # 정확도가 가장 높은 모델을 저장할 변수 선언
    
    for epoch in range(1, num_epochs + 1):
        since = time.time()
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        val_loss, val_acc = evaluate(model, val_loader)
        
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        
        time_elapsed = time.time() - since
        print('-------------- epoch {} ----------------'.format(epoch))
        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))   
        print('val Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    model.load_state_dict(best_model_wts) # 최종적으로 정확도가 가장 높은 모델을 불러온 뒤 반환
    return model

base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)

torch.save(base, 'baseline.pt') # 학습 완료된 모델 저장

-------------- epoch 1 ----------------
train Loss: 1.1099, Accuracy: 67.13%
val Loss: 1.1319, Accuracy: 66.29%
Completed in 0m 11s
-------------- epoch 2 ----------------
train Loss: 0.8432, Accuracy: 73.78%
val Loss: 0.8888, Accuracy: 72.65%
Completed in 0m 11s
-------------- epoch 3 ----------------
train Loss: 0.7068, Accuracy: 77.62%
val Loss: 0.7683, Accuracy: 76.43%
Completed in 0m 11s
-------------- epoch 4 ----------------
train Loss: 0.4928, Accuracy: 84.88%
val Loss: 0.5750, Accuracy: 82.61%
Completed in 0m 11s
-------------- epoch 5 ----------------
train Loss: 0.4233, Accuracy: 87.19%
val Loss: 0.5208, Accuracy: 84.64%
Completed in 0m 11s
-------------- epoch 6 ----------------
train Loss: 0.3906, Accuracy: 87.69%
val Loss: 0.4946, Accuracy: 85.15%
Completed in 0m 11s
-------------- epoch 7 ----------------
train Loss: 0.3374, Accuracy: 89.47%
val Loss: 0.4550, Accuracy: 86.21%
Completed in 0m 11s
-------------- epoch 8 ----------------
train Loss: 0.2895, Accuracy: 91.33%